In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Transaction analysis

This section is devoted to the analysis of the collected Bitcoin TXs.

1. We first collected all Bitcoin TXs with *at least one* dust input and/or dust output.
2. We removed all transactions generated by Satoshi Dice (i.e., those with at least one Satoshi Dice address among their inputs).
3. We classified the remaining TXs into four different groups.
    1. Type 1: TXs with exactly 1 unique input address.
    1. Type 2+: TXs with at least 2 unique input addresses.
    3. Type C: TXs with zero input addresses (coincides with *coinbase* transactions).
    4. Type S: TXs where the sum of all input amounts is equal to the fee.

- From step (1) we discovered that 2114335 TXs have at least one dust input and/or output.
- From step (2) we discovered that 649040 TXs have not been generated by Satoshi Dice.

In [28]:
!wc -l data/txs_dust data/txs_dust_nosd

 2114335 data/txs_dust
  649040 data/txs_dust_nosd
 2763375 total


We classified all 649040 transactions that are not related to Satoshi Dice.

In [38]:
txClass = pd.read_csv('data/txs_dust_nosd_class.csv', header=None)
txClass.columns = ['txId', 'blockId', 'type']
print(f'N. of TXs:\t{len(txClass)}')
txClass.head()

N. of TXs:	649040


,txId,blockId,type
0,121385,82560,1
1,121386,82560,1
2,121403,82561,1
3,121404,82561,1
4,121405,82561,1


- Most of these dust transactions (approximately 65%) belong to the "Type 1" category (i.e., they have exactly 1 unique input address).
- The "Type 2+" category (the most interesting one for successful dust attacks) includes approximately 33% of all TXs.
- There are also 19 transactions classified as "Special", and their inputs are all spent as fees.

In [39]:
pd.DataFrame({'TXs' : txClass.type.value_counts(), 'Percentage' : (txClass.type.value_counts()*100)/len(txClass)})

,TXs,Percentage
1,421094,64.879514
2+,216628,33.376679
C,11299,1.740879
S,19,0.002927


Now we would like to identify the 19 transactions that have been classified as "Special".

In [53]:
txClass[txClass.type == 'S']

,txId,blockId,type
289444,15367109,229265,S
292602,15579990,229814,S
424386,26889044,269132,S
442535,30379718,278511,S
448499,32006969,283392,S
478961,42453637,310571,S
490064,45576189,318089,S
548625,49860416,327218,S
548626,49860421,327218,S
550037,50185095,327766,S


To translate the `txId` to real TX hash, we import the CSV file with the id-to-hash mapping (and then we perform a join on the `txId` attribute of both tables).

In [54]:
txId2Hash = pd.read_csv('data/txId2Hash.csv', header=None)
txId2Hash.columns = ['txId', 'txHash']
txId2Hash

,txId,txHash
0,121385,46b640bcc5395bd6dfde5ec0f2331331b5a4c429e2026a...
1,121386,2b26e886366ec0e08e32866077c46b849e16358f119339...
2,121403,b2f229cf36cf154e254ccc8e21d46ae02fd202a33be892...
3,121404,9d3a11cddc72da6ba0fac7f3a6d370641297db5e329701...
4,121405,85e97e84c5a23063f2afddb03202234f0ee5d7b9a07339...
...,...,...
649035,245354280,d452949aadea80aa64e669565062548a29de00c6ee4f35...
649036,245368332,a0275bc4c4ddf44f569dd67759a07121280760c00b5e61...
649037,245369115,7f1b5d146ec8b8a3a1aabf799a1bcf3ecf69b71feaef62...
649038,245369137,175801edf4fbfd8843f32146f3b05ae3752b6f03494c8c...


In [62]:
special = txClass[txClass.type == 'S'].merge(txId2Hash, how='left', on='txId')
special.to_csv('data/special/special.csv', index=False)
special

,txId,blockId,type,txHash
0,15367109,229265,S,46aa1784c28fb40b0e4351c599d13d6224dfa389273e27...
1,15579990,229814,S,98069c9c8770c4fd1cc73070ee3da07f55284bb62591e8...
2,26889044,269132,S,499e80a173ee095d44b1c3503c5d00015222a2d7c17a21...
3,30379718,278511,S,6164cb05c04d6700ff2cda18f1ba2824d3d02eb782a557...
4,32006969,283392,S,bd5c650f11b0c67365efc6dfb039fbf23f9187455d1e92...
5,42453637,310571,S,52a16a4b7e7376f25151c7362adf3fd5165ae91d013ea6...
6,45576189,318089,S,f482cc27c67f28c037029e45f96b05789e9ca38e7f0ed0...
7,49860416,327218,S,af96ece904c947057e1dee4f84fdca51e277bfd1f58a78...
8,49860421,327218,S,499028aa9984a0d891823a17e0a5c3ab8e8d49b560d9b9...
9,50185095,327766,S,4e59ed6c80aa59e9a3a90b5f655965830cbfb96599a587...


We download the TX details in JSON format.

In [67]:
import requests
import json
for h in special.txHash:
    r = requests.get(url='https://blockchain.info/rawtx/{}'.format(h))
    data = r.json()
    with open('data/special/tx_{}.json'.format(h), 'w') as outfile:
        json.dump(data, outfile)

# Address analysis

(to be completed)

# Input and output analysis

In this section we examine all inputs and outputs of all collected transactions.

In [68]:
inputs = pd.read_csv('data/txs_dust_nosd_inputs.csv')
outputs = pd.read_csv('data/txs_dust_nosd_outputs.csv')

In [69]:
inputs.head()

,timestamp,blockId,txId,address,amount,prevTxId,prevTxOffset
0,1285666089,82560,121385,118901,9988099000,121384,0
1,1285666089,82560,121386,118902,9987098901,121385,1
2,1285666864,82561,121403,118918,9970398802,121402,1
3,1285666864,82561,121404,118919,9969398703,121403,1
4,1285666864,82561,121405,118920,9968398604,121404,1


In [70]:
outputs.head()

,timestamp,blockId,txId,address,amount,scriptType,offset
0,1285666089,82560,121385,118890,99,2,0
1,1285666089,82560,121385,118902,9987098901,2,1
2,1285666089,82560,121386,118890,99,2,0
3,1285666089,82560,121386,118903,9986098802,2,1
4,1285666864,82561,121403,118890,99,2,0


In [71]:
print('N. of inputs:\t{}\nN. of outputs:\t{}'.format(len(inputs),len(outputs)))

N. of inputs:	8594633
N. of outputs:	5497171


How many of these inputs and outputs fall within the dust range $[1, 545]$?

In [76]:
inputs_dust = inputs[(inputs.amount >= 1) & (inputs.amount <= 545)]
outputs_dust = outputs[(outputs.amount >= 1) & (outputs.amount <= 545)]
print('Dust inputs:\t{}\t\t(% over total: {})\nDust outputs:\t{}\t\t(% over total: {})\n'.format(len(inputs_dust), len(inputs_dust)*100/len(inputs), len(outputs_dust), len(outputs_dust)*100/len(outputs)))

Dust inputs:	2559078		(% over total: 29.775302796524297)
Dust outputs:	2935933		(% over total: 53.40807116969801)

